#  UNIVERSIDAD POLITÉCNICA SALESIANA


## Desarrollo 

In [15]:
import numpy
import random
import simpy

# Maximo de vehiculos que puede recibir el negocio
MAX_VEHICULOS     = 57
# Total de maquinas de lavado con que cuenta el negocio
NUM_MAQUINAS      = 3
# Tiempo que tarda en lavarse un vehiculo (minutos)
TIEMPO_LAVADO     = 7 
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 9
# Tiempo de simulación
TIEMPO_SIMULACION = 23

In [16]:
class Lavanderia():
    
    def __init__ (self,enviroment,num_maquinas,tiempo_lavado):
        # Guardamos como variable el entorno de ejecucion
        self.env = enviroment
        # Creamos el recurso que representa las maquinas
        self.maquinas = simpy.Resource(enviroment, num_maquinas)
        # Variable para el tiempo de lavado
        self.tiempo_lavado = tiempo_lavado
     
    def lavar_vehiculo(self,vehiculo):
        # Este metodo representa el proceso de lavado del vehículo.
        # Se ingresa el vehículo y se lava
        
        # Simulamos el tiempo que tarda en lavarse el vehiculo
        # Es importante notar que la instruccion "yield" es distinta de "sleep"
        # ya que esta ultima bloquea el hilo de ejecucion durante 't' unidades de tiempo,
        # mientras que 'yield' no bloquea el hilo de ejecucion, solo lo suspende mientras
        # el evento de 'lavado' se realice
        yield self.env.timeout(TIEMPO_LAVADO)
        print("Lavando el vehículo {}".format(vehiculo))
    
    def llegada_vehiculo(self,vehiculo):
        # Usamos el reloj de la simulacion (env.now()) para indicar a la
        # hora que llega el vehiculo con el nombre pasado como parametro
        print('Llega vehiculo: %s a la hora %.2f.' % (vehiculo, self.env.now))
        # Especificamos que vamos a usar un recurso (Resource) que representa
        # la maquina de lavado
        with self.maquinas.request() as maquina:
            # Ocupamos la maquina de lavado
            yield maquina
            # Indicamos que vehiculo entra a la lavanderia
            print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (vehiculo, env.now))
            # Procesamos la operacion de lavado
            yield env.process(self.lavar_vehiculo(vehiculo))
            # Una vez que termina la llamada con 'yield', se indica que se ha lavado el vehiculo
            print('Vehiculo [%s] lavado a las %.2f.' % (vehiculo, env.now))
    

class Simulacion():
    
    def __init__(self,inicial):
        self.inicial = inicial
    
    def ejecutar_simulacion(self,env, num_maquinas, tiempo_lavado, intervalo):
        lavanderia=Lavanderia(env, num_maquinas, tiempo_lavado)
        self.inicializar_vehiculos(env,lavanderia)
        while True:
            yield env.timeout(random.randint(intervalo-2, intervalo+2))
            self.inicial+=1
            env.process(lavanderia.llegada_vehiculo('Vehiculo-%d'%(self.inicial)))
    
    
    def inicializar_vehiculos(self,env,lavanderia):
        for i in range(self.inicial):
            env.process(lavanderia.llegada_vehiculo('Vehiculo-%d'%(i+1)))


In [17]:
print('*'*10,'Lavanderia UPS','*'*10)
random.seed(77)

# Creamos el entorno de simulacion
env=simpy.Environment()
simulacion = Simulacion(3)
env.process(simulacion.ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))
# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_SIMULACION)

********** Lavanderia UPS **********
Llega vehiculo: Vehiculo-1 a la hora 0.00.
Llega vehiculo: Vehiculo-2 a la hora 0.00.
Llega vehiculo: Vehiculo-3 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 0.00.
Lavando el vehículo Vehiculo-1
Lavando el vehículo Vehiculo-2
Lavando el vehículo Vehiculo-3
Vehiculo [Vehiculo-1] lavado a las 7.00.
Vehiculo [Vehiculo-2] lavado a las 7.00.
Vehiculo [Vehiculo-3] lavado a las 7.00.
Llega vehiculo: Vehiculo-4 a la hora 9.00.
Entra vehiculo a lavarse: Vehiculo-4 a la hora 9.00.
Lavando el vehículo Vehiculo-4
Vehiculo [Vehiculo-4] lavado a las 16.00.
Llega vehiculo: Vehiculo-5 a la hora 18.00.
Entra vehiculo a lavarse: Vehiculo-5 a la hora 18.00.


- Agregar un tiempo de espera entre la llegada del vehiculo y la llegada a la maquina de lavado con un intervalo (1,5)

Procedemos a crear un método que nos permita saber el tiempo que se demorará el vehículo en llegar a la máquina de lavado a partir de su llegada inicial, para ello modificamos la clase **Lavandería()** indicando un número aleatorio de **1 a 5**.

In [18]:
class Lavanderia():
    
    def __init__ (self,enviroment,num_maquinas,tiempo_lavado):
        self.env = enviroment
        self.maquinas = simpy.Resource(enviroment, num_maquinas)
        self.tiempo_lavado = tiempo_lavado
     
    def lavar_vehiculo(self,vehiculo):
        yield self.env.timeout(TIEMPO_LAVADO)
        print("Lavando el vehículo {}".format(vehiculo))
    
    def llegada_al_lavado(self,vehiculo,espera):
        yield self.env.timeout(espera)
    
    def llegada_vehiculo(self,vehiculo):
        with self.maquinas.request() as maquina:
            yield maquina
            
            espera = random.randint(1,5)
            yield env.process(self.llegada_al_lavado(vehiculo,espera))
            
            print('Vehiculo empieza transporte: %s a la hora %.2f.'% (vehiculo, env.now-(espera)))
            print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (vehiculo, env.now))
            
            yield env.process(self.lavar_vehiculo(vehiculo))
            print('Vehiculo [%s] lavado a las %.2f.' % (vehiculo, env.now))

In [19]:
print('*'*10,'UPS','*'*10)
random.seed(77)

env=simpy.Environment()
simulacion = Simulacion(5)
env.process(simulacion.ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))
env.run(until = TIEMPO_SIMULACION)

********** UPS **********
Vehiculo empieza transporte: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 2.00.
Vehiculo empieza transporte: Vehiculo-3 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 2.00.
Vehiculo empieza transporte: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 3.00.
Lavando el vehículo Vehiculo-2
Lavando el vehículo Vehiculo-3
Vehiculo [Vehiculo-2] lavado a las 9.00.
Vehiculo [Vehiculo-3] lavado a las 9.00.
Lavando el vehículo Vehiculo-1
Vehiculo [Vehiculo-1] lavado a las 10.00.
Vehiculo empieza transporte: Vehiculo-4 a la hora 9.00.
Entra vehiculo a lavarse: Vehiculo-4 a la hora 10.00.
Vehiculo empieza transporte: Vehiculo-5 a la hora 9.00.
Entra vehiculo a lavarse: Vehiculo-5 a la hora 12.00.
Vehiculo empieza transporte: Vehiculo-6 a la hora 10.00.
Entra vehiculo a lavarse: Vehiculo-6 a la hora 14.00.
Lavando el vehículo Vehiculo-4
Vehiculo [Vehiculo-4] lavado a las 17.00.
Lavando el vehículo Vehicu

- Identificar el número de máquina dentro de cada proceso
Para saber en que máquina esta el vehículo y también saber en que máquina entra un nuevo vehículo y cual se queda libre necesitamos modificar la clase Lavanderia()

In [20]:
class Lavanderia():
    
    def __init__ (self,enviroment,num_maquinas,tiempo_lavado):
        self.env = enviroment
        self.maquinas = simpy.Resource(enviroment, num_maquinas)
        self.tiempo_lavado = tiempo_lavado
        self.maquinas_uso = {'machine_{}'.format(i+1):False for i in range(num_maquinas)}
    
     
    def lavar_vehiculo(self,vehiculo):
        yield self.env.timeout(TIEMPO_LAVADO)
        print("Lavando el vehículo {}".format(vehiculo))
    
    def llegada_al_lavado(self,vehiculo,espera):
        yield self.env.timeout(espera)
    
    def usar_maquina(self):
        maquina_nombre=''
        for maquina in self.maquinas_uso.items():
            if(maquina[1]==False):
                self.maquinas_uso[maquina[0]]=True
                maquina_nombre=maquina[0]
                break
        return maquina_nombre
    
    def desocupar_maquina(self,key):
        self.maquinas_uso[key]=False
    
    def llegada_vehiculo(self,vehiculo):
        with self.maquinas.request() as maquina:
            yield maquina
            maquina_nombre = self.usar_maquina()
            espera = random.randint(1,5)
            yield env.process(self.llegada_al_lavado(vehiculo,espera))
            print('Vehiculo empieza transporte en la maquina %s: %s a la hora %.2f.'% (maquina_nombre,vehiculo, env.now-(espera)))
            print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (vehiculo, env.now))
            
            yield env.process(self.lavar_vehiculo(vehiculo))
            self.desocupar_maquina(maquina_nombre)
            print('Vehiculo [%s] lavado en la %s a las %.2f.' % (vehiculo,maquina_nombre, env.now))

In [21]:
print('*'*10,'Lavanderia UPS','*'*10)
random.seed(77)

env=simpy.Environment()
simulacion = Simulacion(3)
env.process(simulacion.ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))
env.run(until = TIEMPO_SIMULACION)

********** Lavanderia UPS **********
Vehiculo empieza transporte en la maquina machine_2: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_3: Vehiculo-3 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_1: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 3.00.
Lavando el vehículo Vehiculo-2
Lavando el vehículo Vehiculo-3
Vehiculo [Vehiculo-2] lavado en la machine_2 a las 9.00.
Vehiculo [Vehiculo-3] lavado en la machine_3 a las 9.00.
Lavando el vehículo Vehiculo-1
Vehiculo [Vehiculo-1] lavado en la machine_1 a las 10.00.
Vehiculo empieza transporte en la maquina machine_2: Vehiculo-4 a la hora 9.00.
Entra vehiculo a lavarse: Vehiculo-4 a la hora 10.00.
Lavando el vehículo Vehiculo-4
Vehiculo [Vehiculo-4] lavado en la machine_2 a las 17.00.
Vehiculo empieza transporte en la maquina machine_1: Vehiculo-5 a la hora 17

Ahora finalmente nos falta identificar el tiempo que le toma al vehículo en salir del local una vez que acabó de ser lavado en la máquina.

- Agregar y modificar el tiempo de salir de la máquina a la puerta principal del negocio con un intervalo (2,5)

In [22]:
class Lavanderia():
    
    def __init__ (self,enviroment,num_maquinas,tiempo_lavado):
        self.env = enviroment
        self.maquinas = simpy.Resource(enviroment, num_maquinas)
        self.tiempo_lavado = tiempo_lavado
        self.maquinas_uso = {'machine_{}'.format(i+1):False for i in range(num_maquinas)}
    
     
    def lavar_vehiculo(self,vehiculo):
        yield self.env.timeout(TIEMPO_LAVADO)
        print("Lavando el vehículo {}".format(vehiculo))
    
    def llegada_al_lavado(self,vehiculo,espera):
        yield self.env.timeout(espera)
    
    def salida_local(self,espera):
        yield self.env.timeout(espera)
        
    def usar_maquina(self):
        maquina_nombre=''
        for maquina in self.maquinas_uso.items():
            if(maquina[1]==False):
                self.maquinas_uso[maquina[0]]=True
                maquina_nombre=maquina[0]
                break
        return maquina_nombre
    
    def desocupar_maquina(self,key):
        self.maquinas_uso[key]=False
    
    def llegada_vehiculo(self,vehiculo):
        with self.maquinas.request() as maquina:
            yield maquina
            maquina_nombre = self.usar_maquina()
            espera = random.randint(1,5)
            yield env.process(self.llegada_al_lavado(vehiculo,espera))
            print('Vehiculo empieza transporte en la maquina %s: %s a la hora %.2f.'% (maquina_nombre,vehiculo, env.now-(espera)))
            print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (vehiculo, env.now))
            
            yield env.process(self.lavar_vehiculo(vehiculo))
            self.desocupar_maquina(maquina_nombre)        
            print('Vehiculo [%s] lavado en la %s a las %.2f.' % (vehiculo,maquina_nombre, env.now))
            salida = random.randint(2,5)
            yield env.process(self.salida_local(salida))
            print('Vehiculo empieza transporte a la salida : %s a la hora %.2f.'% (vehiculo, env.now-(salida)))
            print('Vehiculo  sale : %s a la hora %.2f.'% (vehiculo, env.now))

In [23]:
print('*'*10,'Lavanderia UPS','*'*10)
random.seed(77)

env=simpy.Environment()
simulacion = Simulacion(3)
env.process(simulacion.ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))
env.run(until = TIEMPO_SIMULACION)

********** Lavanderia UPS **********
Vehiculo empieza transporte en la maquina machine_2: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_3: Vehiculo-3 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_1: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 3.00.
Lavando el vehículo Vehiculo-2
Lavando el vehículo Vehiculo-3
Vehiculo [Vehiculo-2] lavado en la machine_2 a las 9.00.
Vehiculo [Vehiculo-3] lavado en la machine_3 a las 9.00.
Lavando el vehículo Vehiculo-1
Vehiculo [Vehiculo-1] lavado en la machine_1 a las 10.00.
Vehiculo empieza transporte a la salida : Vehiculo-2 a la hora 9.00.
Vehiculo  sale : Vehiculo-2 a la hora 11.00.
Vehiculo empieza transporte a la salida : Vehiculo-3 a la hora 9.00.
Vehiculo  sale : Vehiculo-3 a la hora 13.00.
Vehiculo empieza transporte a la salida : Vehiculo-1 a la hora 10.00.
V

- Modificamos el tiempo de lavado entre 5 y 11 con valores randómicos

In [24]:
class Lavanderia():
    
    def __init__ (self,enviroment,num_maquinas,tiempo_lavado):
        self.env = enviroment
        self.maquinas = simpy.Resource(enviroment, num_maquinas)
        self.tiempo_lavado = tiempo_lavado
        self.maquinas_uso = {'machine_{}'.format(i+1):False for i in range(num_maquinas)}
    
     
    def lavar_vehiculo(self,vehiculo):
        yield self.env.timeout(random.randint(5,11))
        print("Lavando el vehículo {}".format(vehiculo))
    
    def llegada_al_lavado(self,vehiculo,espera):
        yield self.env.timeout(espera)
    
    def salida_local(self,espera):
        yield self.env.timeout(espera)
        
    def usar_maquina(self):
        maquina_nombre=''
        for maquina in self.maquinas_uso.items():
            if(maquina[1]==False):
                self.maquinas_uso[maquina[0]]=True
                maquina_nombre=maquina[0]
                break
        return maquina_nombre
    
    def desocupar_maquina(self,key):
        self.maquinas_uso[key]=False
    
    def llegada_vehiculo(self,vehiculo):
        with self.maquinas.request() as maquina:
            yield maquina
            maquina_nombre = self.usar_maquina()
            espera = random.randint(1,5)
            yield env.process(self.llegada_al_lavado(vehiculo,espera))
            print('Vehiculo empieza transporte en la maquina %s: %s a la hora %.2f.'% (maquina_nombre,vehiculo, env.now-(espera)))
            print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (vehiculo, env.now))
            
            yield env.process(self.lavar_vehiculo(vehiculo))
            self.desocupar_maquina(maquina_nombre)        
            print('Vehiculo [%s] lavado en la %s a las %.2f.' % (vehiculo,maquina_nombre, env.now))
            salida = random.randint(2,5)
            yield env.process(self.salida_local(salida))
            print('Vehiculo empieza transporte a la salida : %s a la hora %.2f.'% (vehiculo, env.now-(salida)))
            print('Vehiculo  sale : %s a la hora %.2f.'% (vehiculo, env.now))

In [25]:
print('*'*10,'Lavanderia UPS','*'*10)
random.seed(77)
env=simpy.Environment()
simulacion = Simulacion(3)
env.process(simulacion.ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))
env.run(until = TIEMPO_SIMULACION)

********** Lavanderia UPS **********
Vehiculo empieza transporte en la maquina machine_2: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_3: Vehiculo-3 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 2.00.
Vehiculo empieza transporte en la maquina machine_1: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 3.00.
Lavando el vehículo Vehiculo-2
Lavando el vehículo Vehiculo-1
Vehiculo [Vehiculo-2] lavado en la machine_2 a las 8.00.
Vehiculo [Vehiculo-1] lavado en la machine_1 a las 8.00.
Vehiculo empieza transporte a la salida : Vehiculo-2 a la hora 8.00.
Vehiculo  sale : Vehiculo-2 a la hora 12.00.
Lavando el vehículo Vehiculo-3
Vehiculo [Vehiculo-3] lavado en la machine_3 a las 13.00.
Vehiculo empieza transporte a la salida : Vehiculo-1 a la hora 8.00.
Vehiculo  sale : Vehiculo-1 a la hora 13.00.
Vehiculo empieza transporte a la salida : Vehiculo-3 a la hora 13.00.
V